In [ ]:
import ssl
from elasticsearch import Elasticsearch

from dotenv import dotenv_values
config = dotenv_values(".env")

# Elasticsearch Config
es = Elasticsearch(
    ['https://localhost:9200'],
    http_auth=('elastic', config['ELASTIC_PASSWORD']),
    verify_certs=True,
    ca_certs='http_ca.crt'
)

# Check Connection
if not es.ping():
    raise ValueError("Fail to connect to Elasticsearch")
else:
    print("Connect to Elasticsearch Success")


from fastapi import FastAPI, HTTPException
from fastapi.middleware.cors import CORSMiddleware
from elasticsearch import Elasticsearch

# Initialize FastAPI app
PROJECT_NAME="SuiGPT Backend to get relevant_codes"
PROJECT_VERSION="0.0.1"
app = FastAPI(docs_url=None, title=PROJECT_NAME, version = PROJECT_VERSION)

# Enable CORS (Cross-Origin Resource Sharing)
origins = ["*"]  # You can restrict origins as needed
app.add_middleware(
    CORSMiddleware,
    allow_origins=origins,
    allow_credentials=True,
    allow_methods=["*"],
    allow_headers=["*"],
)


from fastapi.openapi.docs import get_swagger_ui_html
from fastapi.openapi.utils import get_openapi

@app.get("/api/docs", include_in_schema=False)
async def get_documentation():
    return get_swagger_ui_html(openapi_url="/api/openapi.json", title="docs")


@app.get("/api/openapi.json", include_in_schema=False)
async def openapi():
    return get_openapi(title = app.title, version=app.version, routes=app.routes)


@app.get("/api/search")
async def get_relevant_codes_by_query(query: str = "A fungible coin name and symbol is Eason that I can manage to airdrop by a list of whitelists"):
    # Define the Elasticsearch query
    query = {
        "query": {
            "more_like_this": {
                "fields": ["code_summary", "code_title"],
                "like": query,
                "min_term_freq": 1,
                "max_query_terms": 12
            }
        }
    }

    # Execute the MLT query
    index_name = "sui_gpt"  # Replace with your Elasticsearch index name
    results = es.search(index=index_name, body=query)

    # Create a dictionary to store the extracted fields
    result_dict = {}

    # Iterate through each document in the Elasticsearch results
    for doc in results['hits']['hits']:
        # Extract the 'code_title' field
        title = doc['_source']['code_title']

        # Extract other fields
        source_move = doc['_source']['source_move']
        annotated_move = doc['_source']['annotated_move']
        annotated_commented_move = doc['_source']['annotated_commented_move']
        code_summary = doc['_source']['code_summary']

        # Store the extracted field values in a dictionary
        result_dict[title] = {
            'source_move': source_move,
            'annotated_move': annotated_move,
            'annotated_commented_move': annotated_commented_move,
            'code_summary': code_summary,
            'code_title': title
        }

    # Take the top 3 items
    top_3_results = dict(list(result_dict.items())[:3])

    return {
        "match_titles": list(result_dict.keys()),
        "top3_match_titles": list(top_3_results.keys()),
        "top3_match": top_3_results,
    }

if __name__ == "__main__":
    import uvicorn
    
    import nest_asyncio
    nest_asyncio.apply()
    
    # Run the FastAPI application
    if "PORT" in config.keys():
        port = config["PORT"]
    else:
        port = 14002
    uvicorn.run(app, host="0.0.0.0", port=port)


/tmp/ipykernel_230461/1893100399.py:8: DeprecationWarning: The 'http_auth' parameter is deprecated. Use 'basic_auth' or 'bearer_auth' parameters instead
  es = Elasticsearch(


Connect to Elasticsearch Success


INFO:     Started server process [230461]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://0.0.0.0:14002 (Press CTRL+C to quit)


INFO:     36.227.201.36:0 - "GET /api/docs HTTP/1.0" 200 OK
INFO:     36.227.201.36:0 - "GET /api/openapi.json HTTP/1.0" 200 OK
INFO:     36.227.201.36:0 - "GET /docs HTTP/1.0" 404 Not Found
INFO:     36.227.201.36:0 - "GET /favicon.ico HTTP/1.0" 404 Not Found
INFO:     36.227.201.36:0 - "GET /favicon.ico HTTP/1.0" 404 Not Found
INFO:     36.227.201.36:0 - "GET /api/search?query=A%20fungible%20coin%20name%20and%20symbol%20is%20Eason%20that%20I%20can%20manage%20to%20airdrop%20by%20a%20list%20of%20whitelists HTTP/1.0" 200 OK
